In [1]:
pip install catboost

     |████████████████████████████████| 64.4MB 65kB/s 


In [2]:
cd sample_data/

/content/sample_data


In [11]:
!unzip Full_data

Archive:  Full_data.zip
  inflating: Full_data.csv           


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#-------------------------------------------------
import lightgbm as lgbm
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import log_loss
#---------------------------------------------------
from sklearn.cluster import KMeans
# from featexp import get_univariate_plots
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import TomekLinks
from sklearn.decomposition import PCA

In [48]:
# Load the data
df = pd.read_csv('Full_data.csv')
Full_data = df
print(df.shape)
df.head()

(67557, 174)


,fid,label,row_loc,col_loc,tile,20190606_B01,20190606_B02,20190606_B03,20190606_B04,20190606_B05,20190606_B06,20190606_B07,20190606_B08,20190606_B8A,20190606_B09,20190606_B11,20190606_B12,20190606_CLD,20190701_B01,20190701_B02,20190701_B03,20190701_B04,20190701_B05,20190701_B06,20190701_B07,20190701_B08,20190701_B8A,20190701_B09,20190701_B11,20190701_B12,20190701_CLD,20190706_B01,20190706_B02,20190706_B03,20190706_B04,20190706_B05,20190706_B06,20190706_B07,20190706_B08,20190706_B8A,...,20190919_CLD,20190924_B01,20190924_B02,20190924_B03,20190924_B04,20190924_B05,20190924_B06,20190924_B07,20190924_B08,20190924_B8A,20190924_B09,20190924_B11,20190924_B12,20190924_CLD,20191004_B01,20191004_B02,20191004_B03,20191004_B04,20191004_B05,20191004_B06,20191004_B07,20191004_B08,20191004_B8A,20191004_B09,20191004_B11,20191004_B12,20191004_CLD,20191103_B01,20191103_B02,20191103_B03,20191103_B04,20191103_B05,20191103_B06,20191103_B07,20191103_B08,20191103_B8A,20191103_B09,20191103_B11,20191103_B12,20191103_CLD
0,2928,4,214,1278,0,0.1995,0.2024,0.1782,0.1898,0.2462,0.3577,0.4018,0.3846,0.4149,0.778,0.3034,0.2589,5.0,0.036,0.0409,0.0706,0.0552,0.1096,0.2826,0.3318,0.3099,0.3572,0.3578,0.2325,0.1307,0.0,0.0361,0.0446,0.0720,0.0588,0.1049,0.2644,0.3296,0.3253,0.3519,...,0.0,0.0419,0.0576,0.0913,0.0941,0.1455,0.2732,0.3229,0.2989,0.3538,0.3444,0.2957,0.1995,0.0,0.049,0.0586,0.0898,0.0711,0.1279,0.2889,0.3427,0.3218,0.3657,0.3573,0.2620,0.1538,0.0,0.0372,0.0536,0.0874,0.0669,0.1239,0.2660,0.3214,0.3204,0.3515,0.361,0.2362,0.1347,0.0
1,2928,4,214,1279,0,0.1995,0.1942,0.1848,0.1846,0.2462,0.3577,0.4018,0.3859,0.4149,0.778,0.3034,0.2589,5.0,0.036,0.0365,0.0682,0.0457,0.1096,0.2826,0.3318,0.3242,0.3572,0.3578,0.2325,0.1307,0.0,0.0361,0.0393,0.0575,0.0419,0.1049,0.2644,0.3296,0.3127,0.3519,...,0.0,0.0419,0.0479,0.0901,0.0784,0.1455,0.2732,0.3229,0.3131,0.3538,0.3444,0.2957,0.1995,0.0,0.049,0.0551,0.0854,0.0673,0.1279,0.2889,0.3427,0.3172,0.3657,0.3573,0.2620,0.1538,0.0,0.0372,0.0457,0.0705,0.0603,0.1239,0.2660,0.3214,0.3296,0.3515,0.361,0.2362,0.1347,0.0
2,2928,4,214,1280,0,0.1995,0.1868,0.1930,0.1860,0.2558,0.3590,0.3982,0.3861,0.4101,0.778,0.3041,0.2560,13.0,0.036,0.0368,0.0504,0.0458,0.1134,0.2615,0.3143,0.3272,0.3525,0.3578,0.2358,0.1311,0.0,0.0361,0.0524,0.0665,0.0637,0.1308,0.2669,0.3313,0.3185,0.3595,...,0.0,0.0419,0.0730,0.0937,0.1028,0.1490,0.2755,0.3264,0.3222,0.3491,0.3444,0.3132,0.2175,0.0,0.049,0.0605,0.0871,0.0802,0.1479,0.2784,0.3277,0.3220,0.3555,0.3573,0.2705,0.1670,0.0,0.0372,0.0565,0.0852,0.0631,0.1259,0.2579,0.3131,0.3459,0.3307,0.361,0.2304,0.1471,0.0
3,2928,4,214,1281,0,0.1995,0.1962,0.1978,0.1954,0.2558,0.3590,0.3982,0.3869,0.4101,0.778,0.3041,0.2560,13.0,0.036,0.0675,0.0881,0.0850,0.1134,0.2615,0.3143,0.2966,0.3525,0.3578,0.2358,0.1311,0.0,0.0361,0.0792,0.1056,0.1036,0.1308,0.2669,0.3313,0.3198,0.3595,...,0.0,0.0419,0.0937,0.1232,0.1358,0.1490,0.2755,0.3264,0.3106,0.3491,0.3444,0.3132,0.2175,0.0,0.049,0.0938,0.1118,0.1202,0.1479,0.2784,0.3277,0.2849,0.3555,0.3573,0.2705,0.1670,0.0,0.0372,0.1022,0.1338,0.1214,0.1259,0.2579,0.3131,0.2861,0.3307,0.361,0.2304,0.1471,0.0
4,2928,4,214,1282,0,0.1995,0.1976,0.1914,0.1956,0.2571,0.3662,0.4005,0.3796,0.4073,0.778,0.3086,0.2573,9.0,0.036,0.0891,0.1258,0.1208,0.1238,0.2837,0.3248,0.3074,0.3602,0.3578,0.2392,0.1517,0.0,0.0361,0.0673,0.1024,0.0872,0.1227,0.2783,0.3413,0.3400,0.3684,...,0.0,0.0419,0.0791,0.1192,0.1248,0.1694,0.2821,0.3247,0.3096,0.3440,0.3444,0.3087,0.2222,0.0,0.049,0.0994,0.1274,0.1090,0.1530,0.2940,0.3447,0.3229,0.3567,0.3573,0.2768,0.1726,0.0,0.0372,0.1164,0.1238,0.1278,0.1385,0.2726,0.3239,0.3286,0.3446,0.361,0.2323,0.1355,0.0


FE :

In [0]:

# NDVI :(B8A-B04)/(B8A+B04)
B8A_features =  df.filter(regex='._B8A')
B8A_features_col = B8A_features.columns
B04_features =  df.filter(regex='._B04')
B04_features_col = B04_features.columns
NDVI = []
for i in range(13) :
    NDVI.append('{}_NDVI'.format(i))
    B8A = '{}'.format(B8A_features_col[i])
    B04 = '{}'.format(B04_features_col[i])
    df['{}_NDVI'.format(i)] = (df[B8A]-df[B04])/(df[B8A]+df[B04])
pca=PCA(1)
df["NDVI"]=pca.fit_transform(df[NDVI])  
df['F_NDVI'] =  df.groupby('NDVI')['fid'].transform('mean')
df['water'] = df['0_NDVI'].apply(lambda x :1 if x<0 else 0)
df['dense_green'] = df['0_NDVI'].apply(lambda x :1 if x>=0.5 else 0)
df['not_green'] = df['0_NDVI'].apply(lambda x :1 if( (x>0) & (x<0.5)) else 0)
# df = df.drop(NDVI,1)

In [0]:
# # SAVI :1.5*(B8A-B04)/(B8A+B04+0.5)
SAVI = []
for i in range(13) :
    SAVI.append('{}_SAVI'.format(i))
    df['{}_SAVI'.format(i)] = 1.5*(Full_data[B8A]-Full_data[B04])/(Full_data[B8A]+Full_data[B04]+0.5)
pca=PCA(1)
df["SAVI"]=pca.fit_transform(df[SAVI])
df['high_green'] = df['0_SAVI'].apply(lambda x :1 if x<0.1 else 0)
df['low_green'] = df['0_SAVI'].apply(lambda x :1 if x>=0.8 else 0)
df = df.drop(SAVI,1)

In [0]:
#RECI :(B08/B04)-1
B08_features =  df.filter(regex='._B08')
B08_features_col = B08_features.columns
RECI = []
for i in range(13) :
    RECI.append('{}_RECI'.format(i))
    B08 = '{}'.format(B08_features_col[i])
    df['{}_RECI'.format(i)] = ((Full_data[B08])/(Full_data[B04]))-1
pca=PCA(1)
df["RECI"]=pca.fit_transform(df[RECI])    
df = df.drop(RECI,1)

In [0]:
columns=['20190606_B04','20190606_B8A', '20190701_B04',
       '20190701_B8A', '20190706_B8A',  '20190711_B04','20190711_B8A', '20190721_B04', '20190721_B8A',
        '20190805_B04', '20190805_B8A','20190815_B04', '20190815_B8A',
      '20190825_B04','20190825_B8A','20190909_B04',  '20190909_B8A',  '20190919_B04', '20190919_B8A',
         '20190924_B04',  '20190924_B8A',
       '20191004_B04','20191004_B8A', '20191103_B04',  '20191103_B8A', ]
data = df[columns].copy()
km=KMeans(5,random_state=2019)
df["cluster_3"]=km.fit_predict(data[columns])

In [0]:
columns_2=df.drop(['fid','label', 'row_loc', 'col_loc', 'tile'],1).columns
data_2 = df[columns_2].copy()
km_2=KMeans(20,random_state=2019)
# df["cluster_1"]=km_2.fit_predict(data_2[columns_2])
df["cluster_2"]=km_2.fit_predict(data_2[columns_2])

In [0]:
# df['SAVI_en'] = df.groupby('SAVI')['cluster_2'].transform('mean')
# df['RECI_en_2'] = df.groupby('NDVI')['cluster_2'].transform('mean')

In [0]:
# MSI :B11/B8A
B11_features =  df.filter(regex='._B11')
B11_features_col = B11_features.columns
MSI = []
for i in range(13) :
    MSI.append('{}_MSI'.format(i))
    B8A = '{}'.format(B8A_features_col[i])
    B11 = '{}'.format(B04_features_col[i])
    df['{}_MSI'.format(i)] = (Full_data[B11])/(Full_data[B8A])
pca=PCA(1)
df["MSI"]=pca.fit_transform(df[MSI])  
df = df.drop('F_NDVI',1)
# df = df.drop(MSI,1)

In [0]:
# NDRE :(B09-B05)/(B09+B05)
B09_features =  df.filter(regex='._B09')
B09_features_col = B09_features.columns
B05_features =  df.filter(regex='._B05')
B05_features_col = B05_features.columns
NDRE = []
for i in range(13) :
    NDRE.append('{}_NDRE'.format(i))
    B09 = '{}'.format(B09_features_col[i])
    B05 = '{}'.format(B05_features_col[i])
    df['{}_NDRE'.format(i)] =  (Full_data[B09]-Full_data[B05])/(Full_data[B09]+Full_data[B05])
pca=PCA(1)
df["NDRE"]=pca.fit_transform(df[NDRE])  
# df = df.drop(NDRE,1)

In [0]:
# ARVI :(B08-(B04-1*(B02-B04)))/(B08+(B04-1*(B02-B04)))
B02_features =  df.filter(regex='._B02')
B02_features_col = B02_features.columns
ARVI = []
for i in range(13) :
    ARVI.append('{}_ARVI'.format(i))
    B08 = '{}'.format(B08_features_col[i])
    B04 = '{}'.format(B04_features_col[i])
    B02 = '{}'.format(B02_features_col[i])
    df['{}_ARVI'.format(i)] = (Full_data[B08]-(Full_data[B04]-1*(Full_data[B02]-Full_data[B04])))/(Full_data[B08]+(Full_data[B04]-1*(Full_data[B02]-Full_data[B04])))
pca=PCA(1)
df["ARVI"]=pca.fit_transform(df[ARVI])  
# df = df.drop(NDRE,1)

In [0]:

# FIDET : B12/(B8A*B09)
B12_features =  df.filter(regex='._B12')
B12_features_col = B11_features.columns
FIDET = []
for i in range(13) :
    FIDET.append('{}_FIDET'.format(i))
    B12 = '{}'.format(B12_features_col[i])
    df['{}_FIDET'.format(i)] =  (Full_data[B12])/(Full_data[B8A]*Full_data[B09])
pca=PCA(1)
df["FIDET"]=pca.fit_transform(df[FIDET])  
# df = df.drop(MSI,1)

In [0]:
df['RECI_en'] = df.groupby('NDVI')['cluster_3'].transform('mean')
df['MSI_en'] = df.groupby('MSI')['cluster_3'].transform('mean')
df['NDRE_en'] = df.groupby('NDRE')['cluster_3'].transform('mean')
df['NDRE_en'] = df.groupby('NDRE')['cluster_3'].transform('mean')
df['FIDET_en'] = df.groupby('FIDET')['cluster_3'].transform('mean')

In [0]:
#EVI :2.5*((B8A-B04)/(B8A+6*B04-7.5*B02+1))
EVI = []
for i in range(13) :
    EVI.append('{}_EVI'.format(i))
    B8A = '{}'.format(B8A_features_col[i])
    B04 = '{}'.format(B04_features_col[i])
    B02 = '{}'.format(B02_features_col[i])
    df['{}_EVI'.format(i)] = 2.5*((Full_data[B8A]-Full_data[B04])/(Full_data[B8A]+6*Full_data[B04]-7.5*Full_data[B02]+1))
pca=PCA(1)
df["EVI"]=pca.fit_transform(df[EVI])  
df['chlorophyll_EVI'] = df['0_EVI'].apply(lambda x :1 if( (x>0.2) & (x<0.8)) else 0)
# df = df.drop(EVI,1)chlorophyll 

In [0]:
#SIPI : (B08-B02)/(B08-B04)
SIPI = []
for i in range(13) :
    SIPI.append('{}_SIPI'.format(i))
    B08 = '{}'.format(B08_features_col[i])
    B04 = '{}'.format(B04_features_col[i])
    B02 = '{}'.format(B02_features_col[i])
    df['{}_SIPI'.format(i)] = (Full_data[B08]-Full_data[B02])/(Full_data[B08]-Full_data[B04])
pca=PCA(1)
# df["SIPI"]=pca.fit_transform(df[SIPI])
df['carotenoids'] = df['0_SIPI'].apply(lambda x :1 if x>0 else 0)
df['chlorophyll-SIPI'] = df['0_SIPI'].apply(lambda x :1 if x<=0 else 0)  
# df = df.drop(SIPI,1)

In [62]:
train.shape

(47791, 287)

Model :

In [63]:
# Split into train and test sets
train = df.loc[df.label != 0]
test =  df.loc[df.label == 0]
train.shape, test.shape
# Split train into train and val, so that we can score our models locally (test is the test set)

# Splitting on field ID since we want to predict for unseen FIELDS
val_field_ids = train.groupby('fid').mean().reset_index()['fid'].sample(frac=0.3).values
tr = train.loc[~train.fid.isin(val_field_ids)].copy()
val = train.loc[train.fid.isin(val_field_ids)].copy()

# Split into X and Y for modelling
X_train, y_train = tr[tr.columns[5:]], tr['label']
X_val, y_val = val[val.columns[5:]], val['label']
# Predicting on fields (grouping first)

# Group
train_grouped = tr.groupby('fid').mean().reset_index()
val_grouped = val.groupby('fid').mean().reset_index()
X_train, y_train = train_grouped[train_grouped.columns[5:]], train_grouped['label']
X_val, y_val = val_grouped[train_grouped.columns[5:]], val_grouped['label']


# Fit model
model =  CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    random_strength=0.1,
    depth=8,
    loss_function='MultiClass'
)
model.fit(X_train.fillna(0), y_train)

# Get predicted probabilities
preds = model.predict_proba(X_val.fillna(0))

# Add to val_grouped dataframe as columns
for i in range(7):
      val_grouped[str(i+1)] = preds[:,i]

# Get 'true' vals as columns in val
for c in range(1, 8):
      val_grouped['true'+str(c)] = (val_grouped['label'] == c).astype(int)

pred_cols = [str(i) for i in range(1, 8)]
true_cols = ['true'+str(i) for i in range(1, 8)]
val_grouped[['label']+true_cols+pred_cols].head()

# Already grouped, but just to double check:
print('Field score:', log_loss(val_grouped.groupby('fid').mean()[true_cols], val_grouped.groupby('fid').mean()[pred_cols]))
# Group test as we did for val
test_grouped = test.groupby('fid').mean().reset_index()
preds = model.predict_proba(test_grouped[train_grouped.columns[5:]])

prob_df = pd.DataFrame({
    'Field_ID':test_grouped['fid'].values
})
for c in range(1, 8):
    prob_df['Crop_ID_'+str(c)] = preds[:,c-1]
prob_df.head()

0:	learn: 1.8633567	total: 1.56s	remaining: 25m 58s
1:	learn: 1.7859124	total: 3.03s	remaining: 25m 11s
2:	learn: 1.7115240	total: 4.52s	remaining: 25m 1s
3:	learn: 1.6162104	total: 5.93s	remaining: 24m 36s
4:	learn: 1.5571744	total: 7.26s	remaining: 24m 4s
5:	learn: 1.4962117	total: 8.61s	remaining: 23m 46s
6:	learn: 1.4565056	total: 9.93s	remaining: 23m 28s
7:	learn: 1.4220450	total: 11.3s	remaining: 23m 18s
8:	learn: 1.3869449	total: 12.6s	remaining: 23m 7s
9:	learn: 1.3583433	total: 14s	remaining: 23m 3s
10:	learn: 1.3319914	total: 15.3s	remaining: 22m 57s
11:	learn: 1.3038857	total: 16.7s	remaining: 22m 51s
12:	learn: 1.2670686	total: 18s	remaining: 22m 46s
13:	learn: 1.2451340	total: 19.3s	remaining: 22m 36s
14:	learn: 1.2251452	total: 20.5s	remaining: 22m 25s
15:	learn: 1.2018027	total: 21.7s	remaining: 22m 13s
16:	learn: 1.1784331	total: 22.9s	remaining: 22m 5s
17:	learn: 1.1608367	total: 24.1s	remaining: 21m 56s
18:	learn: 1.1429572	total: 25.4s	remaining: 21m 49s
19:	learn: 1

,Field_ID,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7
0,3,0.015622,0.958006,0.013572,0.005644,0.001333,0.002051,0.003773
1,6,0.016829,0.976458,0.001004,0.003518,0.001025,0.000946,0.000220
2,11,0.038543,0.725322,0.025072,0.185986,0.008211,0.007907,0.008960
3,13,0.094694,0.874140,0.010119,0.010841,0.005246,0.003706,0.001255
4,14,0.070260,0.753155,0.128786,0.021246,0.019958,0.003240,0.003355


In [64]:
# Check the sample submission and compare
ss = pd.read_csv('SampleSubmission.csv')
ss.head()

,Field_ID,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7
0,3,0,0,0,0,0,0,0
1,6,0,0,0,0,0,0,0
2,11,0,0,0,0,0,0,0
3,13,0,0,0,0,0,0,0
4,14,0,0,0,0,0,0,0


In [65]:
# Merge the two, to get all the required field IDs
ss = pd.merge(ss['Field_ID'], prob_df, how='left', on='Field_ID')
print(ss.isna().sum()['Crop_ID_1']) # Missing fields
# Fill in a low but non-zero val for the missing rows:
ss = ss.fillna(1/7) # There are 34 missing fields
ss.to_csv('Catboost_submission.csv', index=False)

0
